In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

#### data_type (유형별 발매 현황)
- 발매채널별 승차권 발매 현황 : 'channel_sales'
- SR 운영역 승차권 발매현황 : 'srt_station_sales'
- 연도별 SR 승차권 발매현황 : 'yearly_sales'
- 특실/일반실 승차인원 : 'room_class_passengers'

In [2]:
# 총 데이터 수 구하는 함수
def getTotalCnt(data_type, PK):
    url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=1&perPage=10&returnType=JSON&cond[RUN_YMD::LT]=20220101&cond[RUN_YMD::LTE]=20220101&cond[RUN_YMD::GT]=20220101&cond[RUN_YMD::GTE]=20220101'%(data_type, PK)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['totalCount'])
    return cnt

### getAPIconts
---
> __객실 등급별__
> - RUN_YM -> URL의 날짜 설정이 YMD가 아님

> __연도별__
> - 조회 사이트에서 API 링크를 구함(특수기호가 디코드되어 있음)

> __기타__
> - YMD 형식


In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def getAPIconts(data_type, PK, page):
    
    # 최근 일자 구하기(전날)
    yesterday = datetime.today() - timedelta(1) # 가장 최신 일자 (어제)
    yesterday = yesterday.strftime("%Y%m%d")
    
    # 유형별 URL 조합하기
    if data_type == 'room_class_passengers':
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=%s&perPage=10000&returnType=JSON&cond[RUN_YM::LT]=%s&cond[RUN_YM::LTE]=%s&cond[RUN_YM::GT]=20220801&cond[RUN_YM::GTE]=20220801'%(data_type, key,str(page), yesterday, yesterday) 
    elif data_type == 'yearly_sales':
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/yearly_sales?page=1&perPage=10000&cond%5BRUN_YR%3A%3AGTE%5D=2022&cond%5BRUN_YR%3A%3ALTE%5D=2023&serviceKey=B7y6mnXwYiqT8J24zK9IAyF7EGDjLktqZMwR%2F0h8yg6Tq6Scw%2Fj5naZkQIInZ6b2iM%2B6Ndvk%2FFrG4qoTK3gK1Q%3D%3D' 
    else:
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=%s&perPage=10000&returnType=JSON&cond[RUN_YMD::LT]=%s&cond[RUN_YMD::LTE]=%s&cond[RUN_YMD::GT]=20220801&cond[RUN_YMD::GTE]=20220801'%(data_type, key,str(page),yesterday, yesterday)
    
    # API 호출하기
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['data']
    
    return data

### API 가져오기
---
> __발매 채널별 / SR 운영역 별 - 10000개 이상 데이터__ 
>    - 위에서 구한 총 데이터 수량으로 페이지 수 구함

> __연도별 / 객실 등급별__
> - 페이지 1로 고정

> __공통__
>    - 한 페이지 당  10000개 씩 로드
>    - 최근 데이터 : 기준일로부터 이틀 전

In [4]:
# 발매 채널별
channel_sale = []
cnt = getTotalCnt('channel_sales', key)
page = int(cnt/10000) + 1
for i in range(1,page+1):
    js = getAPIconts('channel_sales', key,i)
    for item in js:
        channel_sale.append(item)

# SR 운영역별        
sr_station_sale = []
cnt = getTotalCnt('srt_station_sales', key)
page = int(cnt/10000) + 1
for i in range(1, page+1):
    js = getAPIconts('srt_station_sales', key,i)
    for item in js:
        sr_station_sale.append(item)

# 연도별
yearly_sale = getAPIconts('yearly_sales', key, 1)

# 좌석 등급별
roomclass_sale = getAPIconts('room_class_passengers', key,1)

### 데이터 프레임 변환

In [5]:
channel_sale_df = pd.json_normalize(channel_sale)
sr_station_sale_df = pd.json_normalize(sr_station_sale)
yearly_sale_df = pd.json_normalize(yearly_sale)
roomclass_sale_df = pd.json_normalize(roomclass_sale)
